In [141]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from os import walk
import xlwt 
from xlwt import Workbook 
import tensorflow as tf
from tensorflow import keras
from sklearn.cluster import KMeans

In [142]:
from keras.models import load_model
model = load_model("/content/drive/My Drive/project1/classifier2.h5")

/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [143]:
model.compile(loss="categorical_crossentropy",optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),metrics=["accuracy"])

In [144]:
def detect_white_fg(image_part):
  sayac = 0
  for i in range(0,20,1):
    for j in range(0,20,1):
      point = np.array(image_part[i,j,:]).reshape((3,))

      if ((point[0]<240 and point[0]>225) and (point[1]<240 and point[1]>225)  and (point[2]<240 and point[2]>225)):
        sayac += 1

  if (sayac>110):
    return True
  else:
    return False


In [145]:
def erase_bg(white_fg_c,centroids,hist):

  n,m = centroids.shape
  sayac = 0
  sayac2 = 0

  if (white_fg_c == True):
    for i in range(0,n-1,1):
      j = i+1
      c1 = np.array(centroids[i-sayac2,:]).reshape((3,))
      c2 = np.array(centroids[j-sayac2,:]).reshape((3,))

      if (c1[0]-c2[0]>=0) and (c1[0]-c2[0]>=0) and (c1[0]-c2[0]>=0) and (c1.max()>=220 and c1.sum()>=680) and :
        centroids = np.delete(centroids,j-sayac2,0)
        hist = np.delete(hist,j-sayac2,0)
        sayac2 += 1
      else:
        centroids = np.delete(centroids,i-sayac2,0)
        hist = np.delete(hist,i-sayac2,0)
        sayac2 += 1
  
  else:
    n,m = centroids.shape
    for i in range(0,n,1):
      k,l = centroids.shape
      if (k==1):
        break
      c3 = np.array(centroids[i,:]).reshape((3,))
      if (c3.sum()<=720 and c3.sum()>=680) and (c3.min()>=220):
        centroids = np.delete(centroids,i-sayac,0)
        hist = np.delete(hist,i-sayac,0)
        sayac += 1
        

  centroids = centroids.reshape(-1,3)
  n,m = centroids.shape

  if (n==3):
    centroids = centroids[0:3,:]
    hist = hist[0:3]

  centroids = centroids.reshape(-1,3)
  return (centroids,hist)


In [146]:


def cluster_colors(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    ### VARIABLES
    wn_clusters = 4
    
    ### WHITE FG DETECTION
    image_part = image[60:80,50:70,:]
    white_fg_c = detect_white_fg(image_part)

    ### KMEANS CLUSTERING
    image = image.reshape((image.shape[0] * image.shape[1], 3))

    clt = KMeans(n_clusters = wn_clusters)
    clt.fit(image)

    ### HISTOGRAM

    numLabels = np.arange(0, len(np.unique(clt.labels_)) + 1)
    (hist, _) = np.histogram(clt.labels_, bins = numLabels)
    hist = hist.astype("float")

    ### PLOTTING

    centroids = clt.cluster_centers_

    ### BACKGROUND ELECTION
    centroids,hist = erase_bg(white_fg_c,centroids,hist)

    hist /= hist.sum()

    bar = np.zeros((50, 300, 3), dtype = "uint8")
    startX = 0
    for (percent, color) in zip(hist, centroids):
            # plot the relative percentage of each cluster
            endX = startX + (percent * 300)
            cv2.rectangle(bar, (int(startX), 0), (int(endX), 50),color.astype("uint8").tolist(), -1)
            startX = endX
    return (bar,centroids,hist)

In [147]:
def write_img(image,result,i):
  image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
  
  if result == 0:
    address = "/content/drive/My Drive/rtest3saves/gomlek/"+str(i)+".jpg"
  elif result == 1:
    address = "/content/drive/My Drive/rtest3saves/pantolon/"+str(i)+".jpg"
  elif result == 2:
    address = "/content/drive/My Drive/rtest3saves/t-shirt/"+str(i)+".jpg"
  cv2.imwrite(address,image)
  return address

In [148]:
wb = Workbook()
sheet = wb.add_sheet("Rapor",cell_overwrite_ok=True)

def sheet_write(result,centroid,i,address,hist):
  sheet.write(i+1,0,address)
  if result == 0:
    sheet.write(i+1,1, "gomlek")
  elif result == 1:
    sheet.write(i+1,1, "pantolon")
  elif result == 2:
    sheet.write(i+1,1, "t-shirt")
  (n,m) = centroid.shape
  sayac1=0
  for countit in range(2,(3*2+2),3):
    if (n==1 and sayac1==1):
      sheet.write(i+1,countit,"0")
      sheet.write(i+1,countit+1,"0")
      sheet.write(i+1,countit+2,"0")
    else:
      sheet.write(i+1,countit,str(centroid[sayac1,0]))
      sheet.write(i+1,countit+1,str(centroid[sayac1,1]))
      sheet.write(i+1,countit+2,str(centroid[sayac1,2]))
    sayac1 = sayac1 + 1
  
  sayac2=0
  for countit2 in range(3*2+2,7*2+2,4):
    if (n==1 and sayac2==1):
      sheet.write(i+1,countit2,"0")
      sheet.write(i+1,countit2+1,"0")
      sheet.write(i+1,countit2+2,"0")
      sheet.write(i+1,countit2+3,"0")
    else:
     cmyk_codes = list(rgb_to_cmyk(centroid[sayac2,:]/255))
     sheet.write(i+1,countit2,str(cmyk_codes[0]))
     sheet.write(i+1,countit2+1,str(cmyk_codes[1]))
     sheet.write(i+1,countit2+2,str(cmyk_codes[2]))
     sheet.write(i+1,countit2+3,str(cmyk_codes[3]))
    sayac2 = sayac2 + 1
  if (n==1):
    sheet.write(i+1,7*2+2,"1.0")
    sheet.write(i+1,7*2+3,"0.0")
  else:
    sheet.write(i+1,7*2+2,str(hist[0]))
    sheet.write(i+1,7*2+3,str(hist[1]))
  sheet.write(i+1,7*2+4,str(n))

  



In [149]:
def rgb_to_cmyk(centro):
  k = 1 - centro.max()
  c = (1 - centro[0] - k) / (1 - k)
  m = (1 - centro[1] - k) / (1 - k)
  y = (1 - centro[2] - k) / (1 - k)
  return (c,m,y,k)


In [150]:
path_list = []
for (dirpath, dirnames, filenames) in walk("/content/drive/My Drive/rtest3"):
    path_list.extend(filenames)
    break
results = []
for i in range(0,20,1):
  path = dirpath + '/' + path_list[i]
  image = cv2.imread(path)
  bar,centroid,hist = cluster_colors(image)
  image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
  image = cv2.resize(src=image,dsize=(125,100))
  image = image.reshape((1,100,125,3))
  result = model.predict_classes(image)
  image = image.reshape((100,125,3))
  address = write_img(image,result,i)
  sheet_write(result,centroid,i,address,hist)
  results.extend(result)

In [151]:
wb.save("/content/drive/My Drive/project1/rapor44.xls")